In [2]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('chinook.db')

In [3]:
q = '''
SELECT track.track_id, track.name track_name, artist.name artist_name, med.name track_type, inv.unit_price unit_price, inv.quantity
FROM invoice_line inv
INNER JOIN track ON track.track_id = inv.track_id
INNER JOIN media_type med ON track.media_type_id = med.media_type_id
INNER JOIN album ON album.album_id = track.album_id
INNER JOIN artist ON artist.artist_id = album.artist_id
WHERE inv.invoice_id = 4
;'''

pd.read_sql_query(q, conn)

,track_id,track_name,artist_name,track_type,unit_price,quantity
0,3448,"Lamentations of Jeremiah, First Set \ Incipit ...",The King's Singers,Protected AAC audio file,0.99,1
1,2560,Violent Pornography,System Of A Down,MPEG audio file,0.99,1
2,3336,War Pigs,Cake,Purchased AAC audio file,0.99,1
3,829,Let's Get Rocked,Def Leppard,MPEG audio file,0.99,1
4,1872,Attitude,Metallica,MPEG audio file,0.99,1
5,748,Dealer,Deep Purple,MPEG audio file,0.99,1
6,1778,You're What's Happening (In The World Today),Marvin Gaye,MPEG audio file,0.99,1
7,2514,Spoonman,Soundgarden,MPEG audio file,0.99,1


In [4]:
q = '''
SELECT sub.al_name album, sub.ar_name artist, SUM(il.quantity) tracks_purchased
FROM invoice_line il
INNER JOIN (SELECT t.track_id, ar.name ar_name, al.title al_name
            FROM track t
            INNER JOIN album al ON al.album_id = t.album_id
            INNER JOIN artist ar ON al.artist_id = ar.artist_id) as sub
            ON sub.track_id = il.track_id
GROUP BY album
ORDER BY tracks_purchased DESC
LIMIT 5
;'''

pd.read_sql_query(q, conn)

,album,artist,tracks_purchased
0,Are You Experienced?,Jimi Hendrix,187
1,Faceless,Godsmack,96
2,Mezmerize,System Of A Down,93
3,Get Born,JET,90
4,The Doors,The Doors,83


In [5]:
q = '''
SELECT 
e1.first_name || " " || e1.last_name employee_name,
e1.title employee_title,
e2.first_name || " " || e2.last_name supervisor_name,
e2.title supervisor_title
FROM employee e1
LEFT JOIN employee e2 
ON e1.reports_to = e2.employee_id
ORDER BY employee_name
;'''

pd.read_sql_query(q, conn)

,employee_name,employee_title,supervisor_name,supervisor_title
0,Andrew Adams,General Manager,None,None
1,Jane Peacock,Sales Support Agent,Nancy Edwards,Sales Manager
2,Laura Callahan,IT Staff,Michael Mitchell,IT Manager
3,Margaret Park,Sales Support Agent,Nancy Edwards,Sales Manager
4,Michael Mitchell,IT Manager,Andrew Adams,General Manager
5,Nancy Edwards,Sales Manager,Andrew Adams,General Manager
6,Robert King,IT Staff,Michael Mitchell,IT Manager
7,Steve Johnson,Sales Support Agent,Nancy Edwards,Sales Manager


In [7]:
q = '''
SELECT first_name, last_name, phone
FROM customer
WHERE first_name LIKE "%Belle%"
;'''

pd.read_sql_query(q, conn)

,first_name,last_name,phone
0,Isabelle,Mercier,+33 03 80 73 66 99


In [12]:
q = '''
SELECT 
c.first_name || " " || c.last_name customer_name,
i.purchases number_of_purchases,
i.total_spent,
CASE
    WHEN total_spent < 40 THEN "small spender"
    WHEN total_spent > 100 THEN "big spender"
    ELSE "regular"
END
    AS customer_category
FROM customer c INNER JOIN (SELECT SUM(total) as total_spent, COUNT(*) as purchases, customer_id
                            FROM invoice
                            GROUP BY customer_id
                            ) as i
                ON c.customer_id = i.customer_id
ORDER BY customer_name
;'''

pd.read_sql_query(q, conn)

,customer_name,number_of_purchases,total_spent,customer_category
0,Aaron Mitchell,8,70.29,regular
1,Alexandre Rocha,10,69.30,regular
2,Astrid Gruber,9,69.30,regular
3,Bjørn Hansen,9,72.27,regular
4,Camille Bernard,9,79.20,regular
5,Daan Peeters,7,60.39,regular
6,Dan Miller,12,95.04,regular
7,Diego Gutiérrez,5,39.60,small spender
8,Dominique Lefebvre,9,72.27,regular
9,Eduardo Martins,12,60.39,regular
